# for mimii

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
from sklearn.model_selection import train_test_split

config={"batch_size": 16,"learning_rate": 1e-4,"num_epochs": 20,"resize": 224}

def create_dataset_csv():
    # 数据集根目录
    data_dir = '/media/cheng/data/mimii/mimii-dump'
    data = []

    # 遍历所有子目录
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            # 遍历子目录中的所有wav文件
            for audio_file in os.listdir(label_dir):
                if audio_file.endswith('.wav'):
                    audio_path = os.path.join(label_dir, audio_file)
                    data.append([audio_path, label])

    # 创建DataFrame并保存为CSV
    df = pd.DataFrame(data, columns=['path', 'label'])
    df.to_csv('mimii_dataset.csv', index=False)
    return df

# 自定义数据集类
class AudioDataset(Dataset):
    def __init__(self, df, resize, train_mode=True):
        self.audio_paths = df['path'].values
        # 将标签转换为数值
        self.label_to_idx = {label: idx for idx, label in enumerate(df['label'].unique())}
        self.labels = [self.label_to_idx[label] for label in df['label'].values]
        print(">>> labels  ",self.label_to_idx)
        self.resize = resize
        self.train_mode = train_mode  # 添加训练模式标志
    def __len__(self):
        return len(self.audio_paths)

    def __getitem__(self, idx):
        # 加载音频文件
        waveform, sample_rate = torchaudio.load(self.audio_paths[idx])
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)  # 形状变为 [1, 160000]
          
        # 将音频转换为梅尔频谱图
        transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=2048,
            hop_length=640,
            n_mels=128
        )
        mel_spectrogram = transform(waveform)
        mel_spectrogram = torchaudio.functional.amplitude_to_DB(
                    mel_spectrogram, multiplier=20, amin=1e-10, db_multiplier=0
                )
        # 确保数值在合理范围内
        mel_spectrogram = torch.clamp(mel_spectrogram, min=0)

        # 转换为3通道图像格式 (为了适配ResNet)
        mel_spectrogram = mel_spectrogram.repeat(3, 1, 1)

        # 确保尺寸一致
        resize = torch.nn.AdaptiveAvgPool2d((self.resize, self.resize))
        mel_spectrogram = resize(mel_spectrogram)

        return mel_spectrogram, self.labels[idx]

# 修改ResNet模型
class AudioClassifier(nn.Module):
    def __init__(self, num_classes):
        super(AudioClassifier, self).__init__()
        # 加载预训练的ResNet
        self.resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        # 修改最后的全连接层
        self.resnet.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        return self.resnet(x)

# 训练函数
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            running_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        if epoch%2==0 and epoch!=0:
            torch.save(model.state_dict(),f"resnet18_mimii_{epoch}.pth")
        # config
        if epoch == config['num_epochs']-1:
            torch.save(model.state_dict(),f"resnet18_mimii_last.pth")
        train_loss = running_loss/len(train_loader)
        train_acc = 100.*correct/total

        # 验证阶段
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_loss = val_loss/len(val_loader)
        val_acc = 100.*correct/total

        current_lr = optimizer.param_groups[0]['lr']

        print(f'Epoch {epoch+1}:')
        print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%')
        print(f'Learning Rate: {current_lr:.6f}')

# 主函数
def main():
    # 设置设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    
    # 生成或加载数据集CSV文件
    if not os.path.exists('mimii_dataset.csv'):
        df = create_dataset_csv()
    else:
        df = pd.read_csv('mimii_dataset.csv')

    # 划分训练集和验证集
    train_df = pd.DataFrame()
    val_df = pd.DataFrame()

    for label in df['label'].unique():
        label_df = df[df['label'] == label]
        label_train, label_val = train_test_split(label_df, test_size=0.2, random_state=42)
        train_df = pd.concat([train_df, label_train])
        val_df = pd.concat([val_df, label_val])

    # 创建数据集和数据加载器 
    train_dataset = AudioDataset(train_df, resize=config['resize'], train_mode=True)
    val_dataset = AudioDataset(val_df, resize=config['resize'], train_mode=False)

    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    # 创建模型
    num_classes = len(df['label'].unique())  # 根据实际分类数量设置
    print("num_classes", num_classes)
    model = AudioClassifier(num_classes).to(device)

    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])  

    # 训练模型
    train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=config['num_epochs'], device=device)

if __name__ == "__main__":
    main()

>>> labels   {'abnormal': 0, 'normal': 1}
>>> labels   {'abnormal': 0, 'normal': 1}
num_classes 2
>>> sample_rate   16000
>>> waveform -shape  torch.Size([1, 160000])
>>> sample_rate   16000
>>> waveform -shape  torch.Size([1, 160000])
>>> sample_rate   16000
>>> waveform -shape  torch.Size([1, 160000])
>>> sample_rate   16000
>>> waveform -shape  torch.Size([1, 160000])
>>> sample_rate   16000
>>> waveform -shape  torch.Size([1, 160000])
>>> sample_rate   16000
>>> waveform -shape  torch.Size([1, 160000])
>>> sample_rate   16000
>>> waveform -shape  torch.Size([1, 160000])
>>> sample_rate   16000
>>> waveform -shape  torch.Size([1, 160000])
>>> sample_rate   16000
>>> waveform -shape  torch.Size([1, 160000])
>>> sample_rate   16000
>>> waveform -shape  torch.Size([1, 160000])
>>> sample_rate   16000
>>> waveform -shape  torch.Size([1, 160000])
>>> sample_rate   16000
>>> waveform -shape  torch.Size([1, 160000])
>>> sample_rate   16000
>>> waveform -shape  torch.Size([1, 160000])
>>> s

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
from sklearn.model_selection import train_test_split

config={"batch_size": 16,"learning_rate": 1e-4,"num_epochs": 20,"resize": 224}

def create_dataset_csv():
    # 数据集根目录
    data_dir = '/media/cheng/data/mimii/mimii-dump'
    data = []

    # 遍历所有子目录
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            # 遍历子目录中的所有wav文件
            for audio_file in os.listdir(label_dir):
                if audio_file.endswith('.wav'):
                    audio_path = os.path.join(label_dir, audio_file)
                    data.append([audio_path, label])

    # 创建DataFrame并保存为CSV
    df = pd.DataFrame(data, columns=['path', 'label'])
    df.to_csv('mimii_dataset.csv', index=False)
    return df

# 自定义数据集类
class AudioDataset(Dataset):
    def __init__(self, df, resize, train_mode=True):
        self.audio_paths = df['path'].values
        # 将标签转换为数值
        self.label_to_idx = {label: idx for idx, label in enumerate(df['label'].unique())}
        self.labels = [self.label_to_idx[label] for label in df['label'].values]
        
        self.idx_to_label = {indx:label for label,indx in self.label_to_idx.items()}
        print(">>> label_to_idx  ",self.label_to_idx)
        print(">>> idx_to_label  ",self.idx_to_label)
        self.resize = resize
        self.train_mode = train_mode  # 添加训练模式标志
    def __len__(self):
        return len(self.audio_paths)

    def __getitem__(self, idx):
        # 加载音频文件
        waveform, sample_rate = torchaudio.load(self.audio_paths[idx])
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)  # 形状变为 [1, 160000]
          
        # 将音频转换为梅尔频谱图
        transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=2048,
            hop_length=640,
            n_mels=128
        )
        mel_spectrogram = transform(waveform)
        mel_spectrogram = torchaudio.functional.amplitude_to_DB(
                    mel_spectrogram, multiplier=20, amin=1e-10, db_multiplier=0
                )
        # 确保数值在合理范围内
        mel_spectrogram = torch.clamp(mel_spectrogram, min=0)

        # 转换为3通道图像格式 (为了适配ResNet)
        mel_spectrogram = mel_spectrogram.repeat(3, 1, 1)

        # 确保尺寸一致
        resize = torch.nn.AdaptiveAvgPool2d((self.resize, self.resize))
        mel_spectrogram = resize(mel_spectrogram)

        return mel_spectrogram, self.labels[idx]

# 修改ResNet模型
class AudioClassifier(nn.Module):
    def __init__(self, num_classes):
        super(AudioClassifier, self).__init__()
        # 加载预训练的ResNet
        self.resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        # 修改最后的全连接层
        self.resnet.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        return self.resnet(x)

# 训练函数
def train_model(model, train_loader, val_loader, criterion, optimizer, idx_to_label ,num_epochs=10, device='cpu'):

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            running_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        if epoch%2==0 and epoch!=0:
            torch.save({"labels":idx_to_label ,"weights":model.state_dict()},f"resnet18_mimii_{epoch}.pth")
        # config
        if epoch == config['num_epochs']-1:
            torch.save({"labels":idx_to_label ,"weights":model.state_dict()},f"resnet18_mimii_last.pth")
        train_loss = running_loss/len(train_loader)
        train_acc = 100.*correct/total

        # 验证阶段
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_loss = val_loss/len(val_loader)
        val_acc = 100.*correct/total

        current_lr = optimizer.param_groups[0]['lr']

        print(f'Epoch {epoch+1}:')
        print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%')
        print(f'Learning Rate: {current_lr:.6f}')

# 主函数
def main():
    # 设置设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    
    # 生成或加载数据集CSV文件
    if not os.path.exists('mimii_dataset.csv'):
        df = create_dataset_csv()
    else:
        df = pd.read_csv('mimii_dataset.csv')

    # 划分训练集和验证集
    train_df = pd.DataFrame()
    val_df = pd.DataFrame()

    for label in df['label'].unique():
        label_df = df[df['label'] == label]
        label_train, label_val = train_test_split(label_df, test_size=0.2, random_state=42)
        train_df = pd.concat([train_df, label_train])
        val_df = pd.concat([val_df, label_val])

    # 创建数据集和数据加载器 
    train_dataset = AudioDataset(train_df, resize=config['resize'], train_mode=True)
    val_dataset = AudioDataset(val_df, resize=config['resize'], train_mode=False)
    idx_to_label = train_dataset.idx_to_label 
    
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    # 创建模型
    num_classes = len(df['label'].unique())  # 根据实际分类数量设置
    print("num_classes", num_classes)
    model = AudioClassifier(num_classes).to(device)

    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])  

    # 训练模型
    train_model(model, train_loader, val_loader, criterion, optimizer, idx_to_label ,num_epochs=config['num_epochs'],device=device)

if __name__ == "__main__":
    main()

>>> label_to_idx   {'abnormal': 0, 'normal': 1}
>>> idx_to_label   {0: 'abnormal', 1: 'normal'}
>>> label_to_idx   {'abnormal': 0, 'normal': 1}
>>> idx_to_label   {0: 'abnormal', 1: 'normal'}
num_classes 2
Epoch 1:
Train Loss: 0.4035 | Train Acc: 78.99%
Val Loss: 0.2363 | Val Acc: 91.43%
Learning Rate: 0.000100
Epoch 2:
Train Loss: 0.1104 | Train Acc: 96.14%
Val Loss: 0.2967 | Val Acc: 90.48%
Learning Rate: 0.000100
Epoch 3:
Train Loss: 0.0671 | Train Acc: 97.34%
Val Loss: 0.2013 | Val Acc: 91.43%
Learning Rate: 0.000100
Epoch 4:
Train Loss: 0.0255 | Train Acc: 99.52%
Val Loss: 0.3443 | Val Acc: 90.48%
Learning Rate: 0.000100
Epoch 5:
Train Loss: 0.0191 | Train Acc: 99.52%
Val Loss: 0.2588 | Val Acc: 89.52%
Learning Rate: 0.000100
Epoch 6:
Train Loss: 0.0178 | Train Acc: 99.76%
Val Loss: 0.2986 | Val Acc: 89.52%
Learning Rate: 0.000100
Epoch 7:
Train Loss: 0.0217 | Train Acc: 99.52%
Val Loss: 0.2973 | Val Acc: 91.43%
Learning Rate: 0.000100
Epoch 8:
Train Loss: 0.0154 | Train Acc: 99.7